## 20페이지 자동화

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.common.by import By
import csv

In [ ]:
filename = '블로그140개.csv'
f = open(filename, 'w', encoding='utf-8-sig', newline='')
writer = csv.writer(f)

title = "블로그URL 닉네임 제목 수집텍스트".split()
writer.writerow(title)

# blog_crawl 함수는 블로그 글 하나를 크롤링해서 blog_result에 넣어줌

def blog_crawl():
    blog_result = []
    blog_url = driver.find_element(By.CSS_SELECTOR, '.url._se3copybtn').get_attribute('title')
    nickname = driver.find_element(By.CSS_SELECTOR, 'span.writer > span.nick').text
    blog_title = driver.find_element(By.CSS_SELECTOR, '.se-title-text').text
    contents = driver.find_elements(By.CLASS_NAME, 'se-section-text')
    content_text_list = []
    for content in contents:
        content_text = content.text.replace('\n', '  ')
        content_text_list.append(content_text)
    blog_result = [blog_url, nickname, blog_title, content_text_list]

    return blog_result

final_result = []


# 아래 for문 안에서 총 140개의 블로그 글을 크롤링한다.
# 셀레니움을 이용했으며 웹 드라이버 켜짐 -> 첫번째 글 클릭 -> 그 글이 있는 크롬 탭으로 이동 -> 크롤링 -> 첫번째 크롬 탭으로 이동 -> 두번째 글 클릭 -> 크롤링 -> ... (총 7번 반복)

for page_num in range(1,21):
    print('{}페이지 크롤링 시작합니다.'.format(page_num))
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    time.sleep(4)
    driver.get('https://section.blog.naver.com/Search/Post.naver?pageNo={}&rangeType=PERIOD&orderBy=recentdate&startDate=2023-02-06&endDate=2023-02-08&keyword=%EC%8A%AC%EB%9E%A8%EB%8D%A9%ED%81%AC'.format(page_num))
    time.sleep(4)

    for i in range(1,8):
        element = driver.find_element(By.CSS_SELECTOR, 'div:nth-child({}) > div > div.info_post > div.desc > a.desc_inner'.format(i))
        element.click()
        time.sleep(3)
        driver.switch_to.window(driver.window_handles[i])
        driver.switch_to.frame('mainFrame')

        final_result.extend(blog_crawl())
        writer.writerow(final_result[-4:])
        time.sleep(1)
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(1)